Source: https://www.dataquest.io/blog/pandas-big-data/

In [37]:
import pandas as pd
import numpy as np

In [45]:
movie = pd.read_csv('/Users/ankushchoubey/Downloads/ml-latest-small/movies.csv')
movie.tail()

,movieId,title,genres
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy
9124,164979,"Women of '69, Unboxed",Documentary


In [6]:
movie.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
movieId    9125 non-null int64
title      9125 non-null object
genres     9125 non-null object
dtypes: int64(1), object(2)
memory usage: 1.4 MB


In [8]:
gl_int = movie.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

In [9]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    reply = "{:03.2f} MB".format(usage_mb)
    print(reply)
    return reply

In [15]:
print(mem_usage(gl_int))
print(mem_usage(converted_int))

0.07 MB
0.03 MB


In [16]:
movie[converted_int.columns]= converted_int

In [17]:
movie.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
movieId    9125 non-null uint32
title      9125 non-null object
genres     9125 non-null object
dtypes: object(2), uint32(1)
memory usage: 1.4 MB


In [24]:
gl_obj = movie.select_dtypes(include=['object'])
converted_to_cat = gl_obj['genres'].astype('category')

In [25]:
movie['genres'] = converted_to_cat

In [28]:
movie.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
movieId    9125 non-null uint32
title      9125 non-null object
genres     9125 non-null category
dtypes: category(1), object(1), uint32(1)
memory usage: 912.4 KB


In [70]:
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in.
    conversion = {}
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings

            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)

            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  

            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True


            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    

            # Make float datatypes
            else:
                if mn > np.finfo(np.float16).min and mx < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif mn > np.finfo(np.float32).min and mx < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif mn > np.finfo(np.float64).min and mx < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)   

            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
            conversion[col]=str(df[col].dtype)
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df,conversion

In [71]:
a, NAlist = reduce_mem_usage(movie)

Memory usage of properties dataframe is : 0.17412185668945312  MB
******************************
Column:  movieId
dtype before:  uint32
dtype after:  uint32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  0.17412185668945312  MB
This is  100.0 % of the initial size


In [74]:
ratings_large = pd.read_csv('/Users/ankushchoubey/Downloads/ml-latest/ratings.csv')

In [75]:
d,v,conversion=reduce_mem_usage(ratings_large)

Memory usage of properties dataframe is : 794.1983489990234  MB
******************************
Column:  userId
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  movieId
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  rating
dtype before:  float64
dtype after:  float16
******************************
******************************
Column:  timestamp
dtype before:  int64
dtype after:  uint32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  347.461820602417  MB
This is  43.75000540360532 % of the initial size


ValueError: not enough values to unpack (expected 3, got 2)

In [67]:
conversion

{'movieId': 'uint32',
 'rating': 'float16',
 'timestamp': 'uint32',
 'userId': 'uint32'}